## [ 순환신경망 RNN ] <hr>

In [1]:
import torch
import torch.nn as nn


In [2]:
# ### ===> 데이터 및 초기 Hidden State

# input = torch.randn(1, 3, 10)       # 입력 데이터 (배치크기, 시퀀스길이, 피쳐길이)
# h0 = torch.randn(2, 3, 1)           # 3이 배치사이즈, 1이 히든 초기값

# ### ===> RNN 인스턴스 생성
# rnn = nn.RNN(10, 1, 1)              # 피쳐길이, 히든사이즈, 레이어 수 

# output, hn = rnn(input, h0)

In [3]:
### -------------------------------------------
### ===> 설계 : 다층 RNN, 층 2개
### ===> INPUT
### -------------------------------------------
### 입력 초기 텐서들
input = torch.randn(1, 3, 10)       # 배치사이즈, 시퀀스(1개문장구성 단어 수), 피쳐 수(1개 단어 표현하는 수)
h0 = torch.randn(1, 3, 1)           # 히든스테이트 초기화(양방향*층수, 배치크기, 히든갯수)
### RNN 인스턴스
rnn = nn.RNN(10, 1, 1)

### 출력 텐서들
output, hn = rnn(input, h0)

In [4]:
HIDDEN_SIZE = 1
NUM_LAYERS = 1
SEQ_LENGTH = 3
BATCH_SIZE = 1
BIDIRECTIONAL = 1




### 데이터 및 초기 Hidden state
input = torch.randn(BATCH_SIZE, SEQ_LENGTH, 10)                               # 배치사이즈, 시퀀스(1개문장구성 단어 수), 피쳐 수(1개 단어 표현하는 수)
h0 = torch.randn(NUM_LAYERS*BIDIRECTIONAL, BATCH_SIZE, HIDDEN_SIZE)           # 히든스테이트 초기화(양방향*층수, 배치크기, 히든갯수)
### RNN 인스턴스
rnn = nn.RNN(10, HIDDEN_SIZE, NUM_LAYERS, batch_first=True)

### 출력 텐서들
output, hn = rnn(input, h0)

In [5]:
print(f'[ INPUT DATA ]\n-Shape : {input.shape}     DIM : {input.ndim}D')

[ INPUT DATA ]
-Shape : torch.Size([1, 3, 10])     DIM : 3D


In [6]:
print(f'[ RNN 출력 - output ]\n-Shape : {output.shape}   DIM : {output.ndim}D')

[ RNN 출력 - output ]
-Shape : torch.Size([1, 3, 1])   DIM : 3D


In [7]:
print(f'[ RNN 출력 - hn ]\n-Shape : {hn.shape}   DIM : {hn.ndim}D')

[ RNN 출력 - hn ]
-Shape : torch.Size([1, 1, 1])   DIM : 3D


In [8]:
print(f'[ RNN PARAMETERS]')
for name, param in rnn.named_parameters():
    print(f'-----[{name}]\n{param}\n')

[ RNN PARAMETERS]
-----[weight_ih_l0]
Parameter containing:
tensor([[ 0.5070,  0.9193,  0.3934,  0.0720,  0.7885, -0.9569, -0.0319, -0.5168,
         -0.4571,  0.0134]], requires_grad=True)

-----[weight_hh_l0]
Parameter containing:
tensor([[0.5485]], requires_grad=True)

-----[bias_ih_l0]
Parameter containing:
tensor([0.2836], requires_grad=True)

-----[bias_hh_l0]
Parameter containing:
tensor([-0.2647], requires_grad=True)



In [9]:
### RNN 모델의 속성 출력
print(f'[all_weights] - {rnn.all_weights}, {len(rnn.all_weights)}개')

for name, param in rnn.named_parameters():
    print(f' =====> [{name}]\n{param}\n)')

[all_weights] - [[Parameter containing:
tensor([[ 0.5070,  0.9193,  0.3934,  0.0720,  0.7885, -0.9569, -0.0319, -0.5168,
         -0.4571,  0.0134]], requires_grad=True), Parameter containing:
tensor([[0.5485]], requires_grad=True), Parameter containing:
tensor([0.2836], requires_grad=True), Parameter containing:
tensor([-0.2647], requires_grad=True)]], 1개
 =====> [weight_ih_l0]
Parameter containing:
tensor([[ 0.5070,  0.9193,  0.3934,  0.0720,  0.7885, -0.9569, -0.0319, -0.5168,
         -0.4571,  0.0134]], requires_grad=True)
)
 =====> [weight_hh_l0]
Parameter containing:
tensor([[0.5485]], requires_grad=True)
)
 =====> [bias_ih_l0]
Parameter containing:
tensor([0.2836], requires_grad=True)
)
 =====> [bias_hh_l0]
Parameter containing:
tensor([-0.2647], requires_grad=True)
)


In [10]:
from torchinfo import summary
summary(rnn)

Layer (type:depth-idx)                   Param #
RNN                                      13
Total params: 13
Trainable params: 13
Non-trainable params: 0

In [11]:
### ===> RNN 출력 텐서 output
output.shape, output.ndim, output  # (배치사이즈, 시퀀스, 히든스테이트)


(torch.Size([1, 3, 1]),
 3,
 tensor([[[0.9778],
          [0.9799],
          [0.9716]]], grad_fn=<TransposeBackward1>))

In [12]:
1. 단어 임베딩(Word Embedding)
- 단어 사전: {"i": [1, 0, 0], "am": [0, 1, 0], "here": [0, 0, 1]}

2. 초기 은닉 상태(Initial Hidden State)
- h0 = [0, 0] (모든 값을 0으로 초기화)

3. 시간 단계 t=1 (단어 "i")
- x1 = [1, 0, 0] (단어 "i"의 원핫 벡터)
- h1 = tanh(W_hx * x1 + W_hh * h0 + b_h)
     = tanh([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]] * [1, 0, 0] + [[0.7, 0.8], [0.9, 1.0]] * [0, 0] + [0.1, 0.2])
     = tanh([0.1, 0.2, 0.5, 0.6] + [0.1, 0.2])
     = tanh([0.2, 0.4, 0.6, 0.8])
     = [0.197, 0.378]

4. 시간 단계 t=2 (단어 "am")
- x2 = [0, 1, 0] (단어 "am"의 원핫 벡터)
- h2 = tanh(W_hx * x2 + W_hh * h1 + b_h)
     = tanh([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]] * [0, 1, 0] + [[0.7, 0.8], [0.9, 1.0]] * [0.197, 0.378] + [0.1, 0.2])
     = tanh([0.3, 0.4, 0.5, 0.6] + [0.438, 0.578])
     = tanh([0.738, 0.978, 0.5, 0.6])
     = [0.593, 0.726, 0.462, 0.544]

5. 시간 단계 t=3 (단어 "here")
- x3 = [0, 0, 1] (단어 "here"의 원핫 벡터)
- h3 = tanh(W_hx * x3 + W_hh * h2 + b_h)
     = tanh([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]] * [0, 0, 1] + [[0.7, 0.8], [0.9, 1.0]] * [0.593, 0.726] + [0.1, 0.2])
     = tanh([0.5, 0.6] + [0.915, 1.126])
     = tanh([1.415, 1.726])
     = [0.802, 0.851]

최종 은닉 상태 h3 = [0.802, 0.851]는 RNN의 출력이 됩니다.

SyntaxError: invalid syntax (2245231627.py, line 1)

In [ ]:
네, 원핫 인코딩된 입력부터 계산 과정을 보여드리겠습니다.

1. 입력 시퍼스: [i, am, here, now]

2. 원핫 인코딩
- i: [1, 0, 0, 0]
- am: [0, 1, 0, 0]
- here: [0, 0, 1, 0]
- now: [0, 0, 0, 1]

3. 초기 은닉 상태 
- h0 = [0, 0] (모든 값을 0으로 초기화)

4. 시간 단계 t=1 (단어 "i")
- x1 = [1, 0, 0, 0]
- h1 = tanh(W_hx * x1 + W_hh * h0 + b_h)
     = tanh([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]] * [1, 0, 0, 0] + [[0.9, 1.0], [1.1, 1.2]] * [0, 0] + [0.1, 0.2])
     = tanh([0.1, 0.2, 0.3, 0.4] + [0.1, 0.2])
     = tanh([0.2, 0.4, 0.3, 0.4])
     = [0.197, 0.378, 0.296, 0.378]

5. 시간 단계 t=2 (단어 "am")
- x2 = [0, 1, 0, 0]  
- h2 = tanh(W_hx * x2 + W_hh * h1 + b_h)
     = tanh([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]] * [0, 1, 0, 0] + [[0.9, 1.0], [1.1, 1.2]] * [0.197, 0.378, 0.296, 0.378] + [0.1, 0.2])
     = tanh([0.5, 0.6, 0.7, 0.8] + [0.477, 0.734, 0.522, 0.734])
     = tanh([0.977, 1.334, 1.222, 1.534])
     = [0.728, 0.863, 0.828, 0.907]

6. 시간 단계 t=3 (단어 "here")
- x3 = [0, 0, 1, 0]
- h3 = tanh(W_hx * x3 + W_hh * h2 + b_h)
     = tanh([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]] * [0, 0, 1, 0] + [[0.9, 1.0], [1.1, 1.2]] * [0.728, 0.863, 0.828, 0.907] + [0.1, 0.2])
     = tanh([0.3, 0.4, 0.7, 0.8] + [1.255, 1.603, 1.418, 1.794])
     = tanh([1.555, 2.003, 2.118, 2.594])
     = [0.906, 0.951, 0.958, 0.979]

7. 시간 단계 t=4 (단어 "now")
- x4 = [0, 0, 0, 1]
- h4 = tanh(W_hx * x4 + W_hh * h3 + b_h)
     = tanh([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]] * [0, 0, 0, 1] + [[0.9, 1.0], [1.1, 1.2]] * [0.906, 0.951, 0.958, 0.979] + [0.1, 0.2])
     = tanh([0.4, 0.8] + [1.897, 2.088])
     = tanh([2.297, 2.888])
     = [0.969, 0.989]

최종 은닉 상태 h4 = [0.969, 0.989]가 RNN의 출력이 됩니다.